# PASO 1 – Preparar dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Crear función de preparación con lags y clase
def preparar_dataset_para_regresion(df):
    df = df.sort_values(['product_id', 'periodo'])
    
    # Generar lags tn_1 a tn_11 por producto
    for i in range(1, 12):
        df[f'tn_{i}'] = df.groupby('product_id')['tn'].shift(i)
    
    # Crear campo "clase" con tn en periodo+2
    df['clase'] = df.groupby('product_id')['tn'].shift(-2)
    
    return df

In [8]:
df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
df = df.groupby(['periodo', 'product_id']).agg({'tn' : 'sum'}).reset_index() # Agrupar por periodo y producto
df_prep = preparar_dataset_para_regresion(df)  # Preparar el dataset
df_prep

,periodo,product_id,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,clase
0,201701,20001,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1303.35771
785,201702,20001,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1069.96130
1566,201703,20001,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1502.20132
2352,201704,20001,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1520.06539
3136,201705,20001,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1030.67391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,201701,21295,0.00699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6435,201708,21296,0.00651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
784,201701,21297,0.00579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6436,201708,21298,0.00573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# PASO 2 – Entrenar regresión lineal

In [9]:
# Lista de productos mágicos
magicos = [
    20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
    20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046, 20049,
    20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
    20193, 20320, 20532, 20612, 20637, 20807, 20838
]

# Entrenar regresión lineal en periodo base con productos mágicos
def entrenar_regresion_lineal(df, periodo_base, magicos):
    columnas_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
    
    # Filtrar registros del periodo base y productos mágicos
    df_train = df[(df['periodo'] == periodo_base) & (df['product_id'].isin(magicos))].copy()
    
    # Eliminar filas con datos faltantes
    df_train = df_train.dropna(subset=columnas_tn + ['clase'])

    # Entrenar modelo
    X = df_train[columnas_tn]
    y = df_train['clase']
    model = LinearRegression()
    model.fit(X, y)

    # Mostrar coeficientes
    coef = pd.Series([model.intercept_] + model.coef_.tolist(), index=['intercept'] + columnas_tn)
    return model, coef


# PASO 3 – Predecir con el modelo entrenado

In [26]:
def productos_con_historia():
    df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
    df = df.groupby(['periodo', 'product_id']).agg({'tn' : 'sum'}).reset_index() # Agrupar por periodo y producto
    productos = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')  # Cargar productos
    df = df[df['product_id'].isin(productos['product_id'].unique())].copy()  # Filtrar productos a predecir

    contador = df[(df['periodo']>=201901) & (df['periodo']<=201912) ] 
    contador = contador.groupby('product_id').size().reset_index(name='count')
    product_id = contador[contador['count']==12]['product_id'].unique()

    return product_id

In [ ]:
# df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
# df = df.groupby(['periodo', 'product_id']).agg({'tn' : 'sum'}).reset_index() # Agrupar por periodo y producto
# productos = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')  # Cargar productos
# df = df[df['product_id'].isin(productos['product_id'].unique())].copy()  # Filtrar productos a predecir
# df = df[(df['periodo']>=201901) & (df['periodo']<=201912) ] 
# df = df[df['product_id'].isin(productos_con_historia())]  # Filtrar productos con historia
# df_pred = df[df['periodo'] == 201912].copy() 
# df_pred['tn'].sum()

np.float64(23439.27676)

In [36]:
# Predecir para un periodo futuro
def predecir_regresion(model):
    
    df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
    df = df.groupby(['periodo', 'product_id']).agg({'tn' : 'sum'}).reset_index() # Agrupar por periodo y producto
    productos = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')  # Cargar productos
    df = df[df['product_id'].isin(productos['product_id'].unique())].copy()  # Filtrar productos a predecir
    df = df[(df['periodo']>=201901) & (df['periodo']<=201912) ] 
    df = df[df['product_id'].isin(productos_con_historia())]  # Filtrar productos con historia
    
    df = df.sort_values(['product_id', 'periodo'])
    
    # Generar lags tn_1 a tn_11 por producto
    for i in range(1, 12):
        df[f'tn_{i}'] = df.groupby('product_id')['tn'].shift(i)
    
    columnas_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
    
    
    df_pred = df[df['periodo'] == 201912].copy() 
    

    # Predecir usando regresión lineal
    df_pred['pred'] = model.predict(df_pred[columnas_tn])   
    
    return df_pred

def predecir_no_completos():
    
    df = pd.read_csv('../../data/raw/sell-in.csv', sep='\t')  # Cargar el dataset
    df = df.groupby(['periodo', 'product_id']).agg({'tn' : 'sum'}).reset_index() # Agrupar por periodo y producto
    productos = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')  # Cargar productos
    df = df[df['product_id'].isin(productos['product_id'].unique())].copy()  # Filtrar productos a predecir
    df = df[(df['periodo']>=201901) & (df['periodo']<=201912) ] 
    df = df[~df['product_id'].isin(productos_con_historia())]  # Filtrar productos con historia
    
    df = df.sort_values(['product_id', 'periodo'])
    
    df = df.groupby('product_id').agg({'tn':'mean'}).reset_index()  # Tomar los últimos 12 meses de cada producto
    
    return df

In [46]:
prods = predecir_regresion(modelo)
prods.sort_values(['product_id','periodo'], inplace=True)
prods

,periodo,product_id,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,pred
30316,201912,20001,1504.68856,1397.37231,1561.50552,1660.00561,1261.34529,1678.99318,1109.93769,1629.78233,1647.63848,1470.65653,1259.09363,1275.77351,1162.707525
30317,201912,20002,1087.30855,1423.57739,1979.53635,1090.18771,813.78215,1066.44999,928.36431,1034.98927,1287.62346,1083.62552,1043.01349,1266.78751,1183.640604
30318,201912,20003,892.50129,948.29393,1081.36645,967.77116,635.59563,715.20314,662.38654,590.12515,565.33774,638.04010,758.32657,964.76919,684.763931
30319,201912,20004,637.90002,723.94206,1064.69633,786.17140,482.13372,521.71519,667.19411,603.31081,466.70901,619.77084,441.70332,511.33713,580.484961
30320,201912,20005,593.24443,606.91173,996.78275,879.52808,536.66800,745.74978,876.39696,897.26297,624.99880,488.21387,409.89950,363.58438,563.560780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31229,201912,21248,0.01129,0.02964,0.01270,0.01411,0.02117,0.02116,0.00988,0.01553,0.03106,0.05365,0.06209,0.02962,0.468061
31232,201912,21256,0.01271,0.02682,0.00847,0.00423,0.02965,0.02822,0.00988,0.01553,0.01835,0.05930,0.05081,0.03811,0.463856
31234,201912,21259,0.01412,0.02965,0.01975,0.00564,0.03106,0.04657,0.00988,0.01976,0.02117,0.06777,0.05080,0.04234,0.467856
31235,201912,21262,0.01834,0.02682,0.01693,0.01552,0.02258,0.03953,0.01270,0.01130,0.01412,0.06353,0.05786,0.02680,0.465820


# PASO 4 – Ejecutar todo el flujo

In [39]:
# Paso A: preparar dataset (con tus datos originales en df)
# df_prep = preparar_dataset_para_regresion(df)

# Paso B: entrenar el modelo con periodo 201812
modelo, coeficientes = entrenar_regresion_lineal(df_prep, periodo_base=201812, magicos=magicos)
print("Coeficientes encontrados:")
print(coeficientes)

# Paso C: predecir para registros del periodo 201912 → objetivo: predecir 202002
df_pred_con_historia = predecir_regresion(modelo)
df_pred_con_historia = df_pred_con_historia[['product_id', 'pred']].rename(columns={'pred': 'tn'})

df_pred_sin_historia = predecir_no_completos()

df_pred = pd.concat([df_pred_con_historia, df_pred_sin_historia], ignore_index=True)

# Paso D: ver primeras predicciones
df_pred

Coeficientes encontrados:
intercept    0.441467
tn          -0.001339
tn_1         0.236558
tn_2         0.178208
tn_3        -0.060031
tn_4        -0.161875
tn_5        -0.007775
tn_6         0.151936
tn_7         0.043933
tn_8         0.142839
tn_9         0.103804
tn_10        0.119211
tn_11        0.073671
dtype: float64


,product_id,tn
0,20001,1162.707525
1,20002,1183.640604
2,20003,684.763931
3,20004,580.484961
4,20005,563.560780
...,...,...
775,21252,0.178011
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


# PASO 5 – Exportar a CSV (opcional)

In [40]:
# Exportar archivo para enviar a Kaggle (ajustar columnas si necesario)
df_pred[['product_id', 'tn']].to_csv("./predicciones_regresion_lineal_v1.csv", index=False)

In [67]:
a = pd.read_csv('./outputs/predicciones_regresion_lineal_v1.csv', sep=',')  # Cargar el dataset
b = pd.read_csv('./outputs/predicciones_exp_06_autogluon_v1_12kfold.csv', sep=',')  # Cargar el dataset

a = a.merge(b, on='product_id', how='left')
a['mean'] = a[['tn_x', 'tn_y']].mean(axis=1)
a = a[['product_id', 'mean']].rename(columns={'mean': 'tn'})
a.to_csv("./outputs/predicciones_regresion_lineal_mas_autogluon12kfold.csv", index=False)